### 1.Data Preparation

At first, read the data set

In [1]:
import numpy as np


def readData(filePath: str = "") -> list:
    with open(filePath, 'r+') as f:
        dic = {"data": [], "label": []}
        tmpd = []
        tmpl = []
        for line in f.readlines():
            line = line.strip()
            if line == "-DOCSTART- -X- -X- O":
                continue
            if not line and not tmpd:
                continue
            elif not line and tmpd:
                dic["data"].append(tmpd[:])
                dic["label"].append(tmpl[:])
                tmpd.clear()
                tmpl.clear()
            else:
                word, _, _, tag = line.split(' ')
                tmpd.append(word.lower())
                tmpl.append(tag)
    return dic

Decide padding size: choose the longest sentence in the data set
Find the label nums

In [2]:
testdata, traindata, devdata = readData('./data/test.txt'), readData('./data/train.txt'), readData('./data/valid.txt')

labelset = set()
for word in testdata['label'] + traindata['label'] + devdata['label']:
    for label in word:
        labelset.add(label)
print("The num of labels: ", len(labelset), labelset)

The num of labels:  9 {'B-LOC', 'B-ORG', 'O', 'I-MISC', 'I-LOC', 'B-MISC', 'I-PER', 'I-ORG', 'B-PER'}


In [3]:
import numpy as np


t1 = {}
t2 = {}
for idx, each in enumerate(labelset):
    t1[each] = idx
    t2[idx] = each

t1['PAD'] = len(t1)
t2[len(t2)] = 'PAD'

dataset = {"label2id": t1, "id2label": t2}

def generate_dict(data):
    dic = {'PAD':0}
    for sen in data:
        for w in sen:
            if w not in dic:
                dic[w]=len(dic)
    return dic

# tokenize_func(traindata, dataset['label2id'], dataset['word2id'], 124)
# tokenize_func(devdata, dataset['label2id'], dataset['word2id'], 124)
# tokenize_func(testdata, dataset['label2id'], dataset['word2id'], 124)

dataset["trainset"] = traindata
dataset["devset"] = devdata
dataset["testset"] = testdata
dataset['word2id'] = generate_dict(traindata['data']+devdata['data']+testdata['data'])


write to file

In [4]:
import  json
with open("./Mydata.json",'w+') as f:
    json.dump(dataset,f)

In [ ]:
# loaddata
with open("./Mydata.json",'r+') as f:
    ret = json.load(f)